<a href="https://colab.research.google.com/github/b4wolf/NEURALNETS/blob/development/working_copy_PMWeather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
import json
import os
import pandas as pd

json_folder = 'WI_weather_coords'
csv_folder = 'clean_WI_data'

weather_dict = {}

for filename in os.listdir(json_folder):
    if filename.endswith('.json'):
        with open(os.path.join(json_folder, filename), 'r') as f:
            data = json.load(f)
            prefix = filename.split('.')[0]
            weather_dict[prefix] = data['lat_lon']


json_folder = './WI_weather_coords'
csv_folder = './clean_WI_data'

weather_data_mapping = {}

for json_filename in os.listdir(json_folder):
    if json_filename.endswith('.json'):
        prefix = json_filename.split('.')[0]
        json_path = os.path.join(json_folder, json_filename)
        with open(json_path, 'r') as f:
            data = json.load(f)
            lat_lon = data['lat_lon']

            if prefix not in weather_data_mapping:
                weather_data_mapping[prefix] = {'lat_lon': None, 'csv_path': None}

            weather_data_mapping[prefix]['lat_lon'] = lat_lon

for csv_filename in os.listdir(csv_folder):
    if csv_filename.endswith('.csv'):
        prefix = csv_filename.split('.')[0]

        if prefix in weather_data_mapping:
            csv_path = os.path.join(csv_folder, csv_filename)
            weather_data_mapping[prefix]['csv_path'] = csv_path


lat_lon_mapping = {}

for json_filename in os.listdir(json_folder):
    if json_filename.endswith('.json'):
        prefix = json_filename.split('.')[0]
        json_path = os.path.join(json_folder, json_filename)
        with open(json_path, 'r') as f:
            data = json.load(f)
            lat_lon_mapping[prefix] = data['lat_lon']

weather_data = {}

for csv_filename in os.listdir(csv_folder):
    if csv_filename.endswith('.csv'):
        prefix = csv_filename.split('.')[0]

        if prefix in lat_lon_mapping:
            csv_path = os.path.join(csv_folder, csv_filename)

            df = pd.read_csv(csv_path)

            df['latitude'] = lat_lon_mapping[prefix][0]
            df['longitude'] = lat_lon_mapping[prefix][1]

            weather_data[prefix] = df


In [140]:
import regex as re
directory_path = '/content/'
file_names = [file for file in os.listdir(directory_path) if 'monitor' in file and file.endswith('.csv')]
data_frames = []

for file_name in file_names:
    match = re.match(r'processed_monitor_([0-9.-]+)_([0-9.-]+)\.csv', file_name)
    if match:
        lat, lon = match.groups()
        lat, lon = float(lat), float(lon)
        try:
            df = pd.read_csv(os.path.join(directory_path, file_name))
            df['Latitude'] = lat
            df['Longitude'] = lon
            data_frames.append(df)
        except FileNotFoundError:
            print(f"File not found: {file_name}")
        except pd.errors.EmptyDataError:
            print(f"No data in file: {file_name}")
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")
    else:
        print(f"Filename pattern mismatch: {file_name}")

In [141]:
wet_lat_lon = []
pm_lat_lon = []
for df in weather_data:
  wet_lat_lon.append((weather_data[df]['longitude'][0], weather_data[df]['latitude'][0]))

for index, df in enumerate(data_frames):
  pm_lat_lon.append((df['Latitude'][0], df['Longitude'][0]))

In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c

    return distance

#########################################
# Finds the nearest neighbors between
# weather and PM 2.5 stations
#########################################
def find_nearest_neighbors(list1, list2):
    nearest_neighbors = []

    for point1 in list1:
        min_distance = float('inf')
        nearest_neighbor = None

        for point2 in list2:
            distance = haversine(point1[0], point1[1], point2[0], point2[1])
            if distance < min_distance:
                min_distance = distance
                nearest_neighbor = point2
        nearest_neighbors.append((point1, nearest_neighbor, min_distance))

    return nearest_neighbors

list1 = pm_lat_lon
list2 = wet_lat_lon

nearest_neighbors = find_nearest_neighbors(list1, list2)

for index, n in enumerate(nearest_neighbors):
  nearest_neighbors[index] = (pm_lat_lon.index(n[0]), wet_lat_lon.index(n[1]), n[2])
  data_frames[nearest_neighbors[index][0]] = pd.concat([data_frames[nearest_neighbors[index][0]], weather_data[list(weather_data.keys())[nearest_neighbors[index][1]]]], axis=1)

# cleaning the dataframe to only get the information we need
prepped_data_frames = []
for df in data_frames:
  df = df.drop('Unnamed: 0', axis=1) \
    .drop('latitude', axis=1) \
    .drop('longitude', axis=1) \
    .drop('Index', axis=1) \
    .rename(columns={'Value': 'pm25'})
  prepped_data_frames.append(df)


#########################################
# Finds edges between stations
#########################################

G = nx.Graph()
distances = {}
node_positions = {}

for i in range(len(data_frames)):
    coord1 = (prepped_data_frames[i]['Latitude'].iloc[0], prepped_data_frames[i]['Longitude'].iloc[0])
    node_positions[i] = coord1
    distances[i] = []

    for j in range(len(data_frames)):
        if i != j:
            coord2 = (prepped_data_frames[j]['Latitude'].iloc[0], prepped_data_frames[j]['Longitude'].iloc[0])
            distance = haversine(coord1[0], coord1[1], coord2[0], coord2[1])
            distances[i].append((distance, j))

for i, dist_list in distances.items():
    three_nearest = nsmallest(3, dist_list)
    for dist, j in three_nearest:
        G.add_edge(i, j, weight=dist)

plt.figure(figsize=(10, 8))
nx.draw(G, pos=node_positions, with_labels=True, node_size=50, node_color="skyblue", edge_color="gray")
plt.title("Station Graph")
plt.show()
list(G.edges())

In [150]:
adj_mat = nx.adjacency_matrix(G)

In [ ]:
!pip install torch_geometric

In [152]:
import torch
import folium
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from scipy.spatial.distance import euclidean
import os
import networkx as nx
from torch_geometric.data import Data
import math
from heapq import nsmallest
import pandas as pd
import re
import glob
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import scipy.sparse as sparse

In [155]:
sparse.csr_matrix(adj_mat)

<11x11 sparse matrix of type '<class 'numpy.float64'>'
	with 46 stored elements in Compressed Sparse Row format>

In [157]:
stat_train_predictions = []
stat_test_predictions = []

for df in prepped_data_frames:
  timeseries = df.drop('Latitude', axis=1).drop('Longitude', axis=1).values.astype('float32')
  train_size = int(len(timeseries) * 0.70)
  test_size = len(timeseries) - train_size
  train, test = timeseries[:train_size], timeseries[train_size:]

  def create_dataset(dataset, lookback):
      """Transform a time series into a prediction dataset

      Args:
          dataset: A numpy array of time series, first dimension is the time steps
          lookback: Size of window for prediction
      """
      X, y = [], []
      for i in range(len(dataset)-lookback):
          feature = dataset[i:i+lookback]
          target = dataset[i+1:i+lookback+1]
          X.append(feature)
          y.append(target)
      return torch.tensor(X), torch.tensor(y)

  lookback = 7
  X_train, y_train = create_dataset(train, lookback=lookback)
  X_test, y_test = create_dataset(test, lookback=lookback)
  class AirModel(nn.Module):
      def __init__(self):
          super().__init__()
          self.lstm = nn.LSTM(input_size=9, hidden_size=50, num_layers=1, batch_first=True)
          self.linear = nn.Linear(50, 9)
      def forward(self, x):
          x, _ = self.lstm(x)
          x = self.linear(x)
          return x

  model = AirModel()
  optimizer = optim.Adam(model.parameters())
  loss_fn = nn.MSELoss()
  loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=8)

  n_epochs = 100
  for epoch in range(n_epochs):
      model.train()
      losses = []
      for X_batch, y_batch in loader:
          y_pred = model(X_batch)
          loss = loss_fn(y_pred, y_batch)
          losses.append(loss.detach().numpy())
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
      # Validation
      model.eval()
      with torch.no_grad():
          y_pred = model(X_train)
          train_rmse = np.sqrt(loss_fn(y_pred, y_train))
          y_pred = model(X_test)
          test_rmse = np.sqrt(loss_fn(y_pred, y_test))
      print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch, train_rmse, test_rmse))

  with torch.no_grad():
    y_pred = model(X_train)
    stat_train_predictions.append(y_pred[:, -1, :])
    y_pred = model(X_test)
    stat_test_predictions.append(y_pred[:, -1, :])

  """with torch.no_grad():
      # shift train predictions for plotting
      train_plot = np.ones_like(timeseries) * np.nan
      y_pred = model(X_train)
      y_pred = y_pred[:, -1, :]
      print(y_pred.shape)
      train_plot[lookback:train_size] = y_pred[:, 8]

      # shift test predictions for plotting
      test_plot = np.ones_like(timeseries) * np.nan
      test_plot[train_size+lookback:len(timeseries)] = model(X_test)[:, -1, 8]"""

"""  # plot
  plt.plot(timeseries[:, 8])
  plt.plot(train_plot, c='r')
  plt.plot(test_plot, c='g')
  plt.show()"""

Epoch 0: train RMSE 59.4899, test RMSE 58.6489
Epoch 1: train RMSE 51.1951, test RMSE 50.5570
Epoch 2: train RMSE 45.8863, test RMSE 45.2804
Epoch 3: train RMSE 41.8865, test RMSE 41.2131
Epoch 4: train RMSE 38.6746, test RMSE 37.9637
Epoch 5: train RMSE 36.0135, test RMSE 35.2367
Epoch 6: train RMSE 33.7814, test RMSE 32.9782
Epoch 7: train RMSE 31.7209, test RMSE 30.8539
Epoch 8: train RMSE 30.1168, test RMSE 29.3411
Epoch 9: train RMSE 28.8104, test RMSE 28.0957
Epoch 10: train RMSE 27.7456, test RMSE 27.1332
Epoch 11: train RMSE 26.9281, test RMSE 26.5209
Epoch 12: train RMSE 26.4243, test RMSE 26.0660
Epoch 13: train RMSE 25.8736, test RMSE 25.8399
Epoch 14: train RMSE 25.4926, test RMSE 25.5892
Epoch 15: train RMSE 25.1989, test RMSE 25.3809
Epoch 16: train RMSE 24.9736, test RMSE 25.2179
Epoch 17: train RMSE 24.7795, test RMSE 25.0549
Epoch 18: train RMSE 24.5762, test RMSE 25.0146
Epoch 19: train RMSE 24.4911, test RMSE 25.0268
Epoch 20: train RMSE 24.5016, test RMSE 25.0141
Ep

KeyboardInterrupt: ignored

In [ ]:
df_tensor_11_train = pd.DataFrame(torch.cat(tuple(stat_train_predictions), axis=1).numpy())
df_tensor_11_test = pd.DataFrame(torch.cat(tuple(stat_test_predictions), axis=1).numpy())

In [ ]:
edge_index = torch.tensor(list(G.edges())).t().contiguous()

# change x to have stat train and test predictions
x = torch.tensor([[df_tensor_11_train['Value'].values(), ] for df in data_frames], dtype=torch.float)

# this one is the targets, which I think is right rn
targets = torch.tensor([df[['Value']].values for df in data_frames], dtype=torch.float)
data = Data(x=x, edge_index=edge_index, y=targets)
num_nodes = data.num_nodes
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_size = int(num_nodes * 0.8)
train_mask[:train_size] = True
test_mask[train_size:] = True
data.train_mask = train_mask
data.test_mask = test_mask

class BasicGNN(torch.nn.Module):
    def __init__(self, num_features):
        super(BasicGNN, self).__init__()
        self.conv1 = SAGEConv(num_features, 16)
        self.conv2 = SAGEConv(16, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x.squeeze()

model = BasicGNN(num_features=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

for epoch in range(2000):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()